Generation de data_tr_lemm.csv
- Prend data_tr en entrée (= traduction de data.csv)

In [1]:
import os

# Désactiver le GPU en définissant CUDA_VISIBLE_DEVICES à un vide#
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer


from RktnChallenge.preprocessing.tokenizeString import tokenizeString
from RktnChallenge.preprocessing.filterStopWords import filterStopWords
from RktnChallenge.preprocessing.mergeFeatures import mergeFeatures
from RktnChallenge.preprocessing.mostOccur import mostOccur
from RktnChallenge.preprocessing.Dropper import Dropper
from RktnChallenge.preprocessing.TokenListToString import TokenListToString



import RktnChallenge.preprocessing.filterChar
importlib.reload(RktnChallenge.preprocessing.filterChar)
from RktnChallenge.preprocessing.filterChar import filterChar

import RktnChallenge.preprocessing.lemmatize
importlib.reload(RktnChallenge.preprocessing.lemmatize)
from RktnChallenge.preprocessing.lemmatize import lemmatize

import RktnChallenge.preprocessing.regularExprSub
importlib.reload(RktnChallenge.preprocessing.regularExprSub)
from RktnChallenge.preprocessing.regularExprSub import regularExprSub

2023-10-02 10:48:14.060112: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-02 10:48:14.084431: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9511] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-02 10:48:14.084455: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-02 10:48:14.084470: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-02 10:48:14.089576: I tensorflow/core/platform/cpu_feature_guar

In [2]:
RktnModel = ModelTrainer("data_tr_08_.csv")
nom_sauvegarde = "data_tr_lemm_.csv"
nom_sauvegarde_sans_lemm = "data_tr_brut.csv"

Process Complet, si l'on veut tester avec lemmatization

In [3]:
addendum = ['a','à','<p>','<b>','<div>','<em>','<br>','gt',
            'h','b','k','a','c','d','e','ex','Ndeg']

filtr = ['+','-','/','(',')',':',"'",'"',':','@''!','|','#','<','>','?',
         '1','2','3','4','5','6','7','8','9','0']

#on tokenise et retire les stopwords designation

desi_filter = filterChar("designation","tr_designation", filtr)

desi_tokenize= tokenizeString("designation_tkn","designation")
desi_stopwords = filterStopWords("designation_tkn", "designation_tkn"
                                 ,lang=["english","french"],addendum=addendum)

desi_lemmatize = lemmatize("designation_tkn","designation_tkn")

#on tokenise et retire les stopwords description
desc_filter = filterChar("description","tr_description", filtr)
desc_tokenize= tokenizeString("description_tkn","description")
desc_stopwords = filterStopWords("description_tkn", "description_tkn"
                                 ,lang=["english","french"],addendum=addendum)

desc_lemmatize = lemmatize("description_tkn","description_tkn")
merge_desi_desc = mergeFeatures("merged","designation_tkn","description_tkn")
desc_mostOccur= mostOccur("merged_desi_desc","merged",1000,1000)



toDrop = ["designation_tkn","description_tkn","merged"]
cleanDropper = Dropper(column_to_drop = toDrop)
toString = TokenListToString("merged_desi_desc","merged_desi_desc")



RktnModel.initPreprocess()
RktnModel.addPreprocessStep("clean designation",desi_filter) # filtre tous les caracteres et mots incorrects
RktnModel.addPreprocessStep("tokenize designation",desi_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter designation",desi_stopwords)  # filtrage des stopwords
RktnModel.addPreprocessStep("lemmitize designation",desi_lemmatize)  # Lemmatisation designation

RktnModel.addPreprocessStep("clean description",desc_filter) # filtre tous les caracteres et mots incorrects
RktnModel.addPreprocessStep("tokenize description",desc_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter description",desc_stopwords)  # filtrage des stopwords
RktnModel.addPreprocessStep("lemmitize description",desc_lemmatize)  # Lemmatisation
RktnModel.addPreprocessStep("merge designation and desc",merge_desi_desc) # merge desi&desc en un seul champ
RktnModel.addPreprocessStep("desc reduce",desc_mostOccur)         # reduction taille desc
RktnModel.addPreprocessStep("tostring",toString) # convertit en chaine de caracteres


#RktnModel.addPreprocessStep("cleaning",cleanDropper) # on clean les colonnes crées
RktnModel.preprocess()

removeNumbers = regularExprSub("merged_desi_desc","merged_desi_desc",filtr='\d+')
RktnModel.initPreprocess()
RktnModel.addPreprocessStep("removeNumbers",removeNumbers) # on enleve les nombres qui pourraient rester
RktnModel.preprocess()

RktnModel.saveCSV(nom_sauvegarde)

[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Tentative de sauvegarde avec et sans lemmatization, et crop sur description

In [6]:
addendum = ['a','à','<p>','<b>','<div>','<em>','<br>','gt',
            'h','b','k','a','c','d','e','ex','-','ndeg','(',')','r',".",'"',"*",'/',"ndeg1","ndgeg126","!",":","ndeg111"]

filtr = ['+','-','/','(',')',"'",'"',':','@','!','|','#','<','>','?']

lang = ['english','french', 
 ]



#on tokenise et retire les stopwords designation

desi_tokenize= tokenizeString("designation_tkn","tr_designation")
desi_stopwords = filterStopWords("designation_tkn", "designation_tkn"
                                 ,lang=lang,addendum=addendum)


#on tokenise et retire les stopwords description

desc_tokenize= tokenizeString("description_tkn","tr_description")
desc_stopwords = filterStopWords("description_tkn", "description_tkn"
                                 ,lang=lang,addendum=addendum)



merge_desi_desc = mergeFeatures("merged","designation_tkn","description_tkn")
desc_mostOccur= mostOccur("merged_desi_desc","merged",700,1000)

toString = TokenListToString("merged_desi_desc","merged_desi_desc")



RktnModel.initPreprocess()
#RktnModel.addPreprocessStep("clean designation",desi_filter) # filtre tous les caracteres et mots incorrects
RktnModel.addPreprocessStep("tokenize designation",desi_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter designation",desi_stopwords)  # filtrage des stopwords
#RktnModel.addPreprocessStep("lemmitize designation",desi_lemmatize)  # Lemmatisation designation

#RktnModel.addPreprocessStep("clean description",desc_filter) # filtre tous les caracteres et mots incorrects
RktnModel.addPreprocessStep("tokenize description",desc_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter description",desc_stopwords)  # filtrage des stopwords
#RktnModel.addPreprocessStep("lemmitize description",desc_lemmatize)  # Lemmatisation
RktnModel.addPreprocessStep("merge designation and desc",merge_desi_desc) # merge desi&desc en un seul champ



RktnModel.addPreprocessStep("desc reduce",desc_mostOccur)         # reduction taille desc
RktnModel.addPreprocessStep("tostring",toString) # convertit en chaine de caracteres

RktnModel.preprocess()

RktnModel.saveCSV(nom_sauvegarde)

[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
display(RktnModel.data.head())



,Unnamed: 0.1,Unnamed: 0,designation,description,productid,imageid,prdtypecode,imgname,desi_langue,tr_designation,desc_langue,tr_description,designation_tkn,description_tkn,merged,merged_desi_desc
0,0,0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10,image_1263597046_product_3804725264.jpg,fr,Olivia: Personalisiertes Notizbuch / 150 Seite...,fr,NaN,"[olivia, personalisiertes, notizbuch, 150, sei...",[],"[olivia, personalisiertes, notizbuch, 150, sei...",olivia personalisiertes notizbuch 150 seiten p...
1,1,1,Journal Des Arts (Le) Ndeg 133 Du 28/09/2001 -...,NaN,436067568,1008141237,2280,image_1008141237_product_436067568.jpg,fr,Journal Des Arts (Le) Ndeg 133 Du 28/09/2001 -...,fr,NaN,"[journal, arts, 133, 28, 09, 2001, l', art, ma...",[],"[journal, arts, 133, 28, 09, 2001, l', art, ma...",journal arts 133 28 09 2001 l' art marche salo...
2,2,2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50,image_938777978_product_201115110.jpg,fr,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,fr,PILOT STYLE Touch Pen de marque Speedlink est ...,"[grand, stylet, ergonomique, bleu, gamepad, ni...","[pilot, style, touch, pen, marque, speedlink, ...","[grand, stylet, ergonomique, bleu, gamepad, ni...",grand stylet ergonomique bleu gamepad nintendo...
3,3,3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280,image_457047496_product_50418756.jpg,fr,Peluche Donald - Europe - Disneyland 2000 (Mar...,fr,NaN,"[peluche, donald, europe, disneyland, 2000, ma...",[],"[peluche, donald, europe, disneyland, 2000, ma...",peluche donald europe disneyland 2000 marionne...
4,4,4,La Guerre Des Tuques,Luc a des idees de grandeur. Il veut organiser...,278535884,1077757786,2705,image_1077757786_product_278535884.jpg,fr,La Guerre Des Tuques,fr,Luc a des idees de grandeur. Il veut organiser...,"[guerre, tuques]","[luc, idees, grandeur, veut, organiser, jeu, g...","[guerre, tuques, luc, idees, grandeur, veut, o...",guerre tuques luc idees grandeur veut organise...


In [ ]:
Calcul des tailles des champs texte

In [2]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer

import pandas as pd
nom_sauvegarde="embeddings_final.csv"
RktnModel = ModelTrainer(nom_sauvegarde)
RktnModel.data['description_length'] = RktnModel.data['description'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
RktnModel.data['designation_length'] = RktnModel.data['designation'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)

min_val = RktnModel.data['description_length'].min()
max_val = RktnModel.data['description_length'].max()
RktnModel.data['description_length_normalized'] = (RktnModel.data['description_length'] - min_val) / (max_val - min_val)

min_val = RktnModel.data['designation_length'].min()
max_val = RktnModel.data['designation_length'].max()
RktnModel.data['designation_length_normalized'] = (RktnModel.data['designation_length'] - min_val) / (max_val - min_val)

RktnModel.saveCSV(nom_sauvegarde)


2023-09-25 11:11:53.943345: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-25 11:11:53.966277: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9511] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-09-25 11:11:53.966299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-09-25 11:11:53.966312: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-25 11:11:53.971373: I tensorflow/core/platform/cpu_feature_guar